## CAT DOG CLASSIFICATION USING SVM


In [1]:
import glob
import random
from pathlib import Path
import pickle
import numpy as np 
import pandas as pd 
import shutil

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [50]:
from skimage.feature import hog
from skimage import exposure
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## FEATURE EXTRACTION

In [29]:
def extract_hog_features(file):
    image = cv2.imread(file, 0)  # Read the image in grayscale
    resized_image = cv2.resize(image, (64, 128))  # Resize image to a common size
    features, hog_image = hog(resized_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=True)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return features 

In [36]:
df = pd.DataFrame(columns=['Path', 'Class'])

for pet_class in ['Cat', 'Dog']:
    for path in glob.glob('train1/' + pet_class + '/*'):
        df.loc[len(df)] = [path, pet_class]

In [37]:
lbl_encoder = LabelEncoder()
df['Class'] = lbl_encoder.fit_transform(df['Class'])

In [38]:
X_train_paths, X_test_paths, y_train, y_test = train_test_split(df['Path'], df['Class'], test_size=0.2, random_state=101)

In [39]:
X_train = [extract_hog_features(path) for path in X_train_paths]
X_test = [extract_hog_features(path) for path in X_test_paths]

In [40]:
X_train = np.array(X_train)
X_test = np.array(X_test)

## SVM MODEL

In [42]:
svc = SVC()
model = svc.fit(X_train, y_train)

# Evaluate the SVC
accuracy = model.score(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.7047817047817048


In [43]:
filename = "SVC_model.sav"
pickle.dump(model, open(filename, 'wb'))

In [41]:
print(f"The Classification Report for SVC model is \n{classification_report(y_test,predictions)}")

The Classification Report for SVC model is 
              precision    recall  f1-score   support

           0       0.54      0.57      0.56       240
           1       0.55      0.52      0.54       241

    accuracy                           0.55       481
   macro avg       0.55      0.55      0.55       481
weighted avg       0.55      0.55      0.55       481



In [44]:
df_test = pd.DataFrame(columns=['id', 'Path'])

for path in glob.glob('test1/*/*'):
    # Extract image ID using the file name (assuming the file name contains the ID)
    image_id = int(path.split('\\')[-1].split('.')[0])
    df_test.loc[len(df_test)] = [image_id, path]

In [45]:
X_test_final = [extract_hog_features(path) for path in df_test['Path']]

In [46]:
X_test_final = np.array(X_test_final)

In [47]:
predictions = svc.predict(X_test_final)

In [48]:
submission_df = pd.DataFrame({'id': df_test['id'], 'label': predictions})

submission_df.to_csv('submission.csv', index=False)